<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/capitaliza%C3%A7%C3%A3o%20one%20mar%2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
capit="Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb"
folha="listagem upfront"
folha_pagar="Valor a pagar"

In [4]:
#carregar ficheiro dos alarmes

from google.colab import files
uploaded = files.upload()

Saving Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb to Copy of unit_03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas_VS3_2 - TESTE.xlsb


In [5]:
#ler folha auto

folha_capit=pd.read_excel(capit, sheet_name=folha)

In [8]:
#ler folha valor a pagar

folha_pagar=pd.read_excel(capit, sheet_name=folha_pagar)

In [7]:
#definir lista com o nome das colunas

colunas_pagar=["username", "chefe de equipa", "função", "parceiro", "Activações", "Correção autos anteriores", "Correções autos anteriores_histórico",\
"Correções autos anteriores a aplicar", "Total ativações", "Penalidades N", "Penalidades_histórico", "Penalidade total a aplicar", \
"Charge back N", "Charge back_histórico", "Charge back total a aplicar", "Ativação Negativa", "Ativação Negativa Histórico", "Ativação Negativa total a aplicar", \
"Melhoria Contínua", "Valor", "VALOR COM FATOR", "Correções a transitar", "Penalidades a transitar", "Charge back a transitar", "Ativações negativas a transitar", \
"Status CAV N-1", "Status CAV N", "Imputar ao CC do OBV_correções", "Imputar ao CC do OBV_penalidades", "Imputar ao CC do OBV_charge back", \
"Imputar ao CC do OBV_ativ neg", "Correções a transitar para o mês seguinte", "Penalidade a transitar para o mês seguinte", \
"Charge back a transitar para o mês seguinte", "Ativações negativas a transitar para o mês seguinte", "rede", "Auto", "CAV", "Valores_transitar", \
"Comentário"]
print(len(colunas_pagar))

40


In [9]:
#definir uma nova bd
folhas_pagar2=folha_pagar.iloc[:, :len(colunas_pagar)].dropna().iloc[1:, :]
folhas_pagar2.columns=colunas_pagar
print(folhas_pagar2.shape)
print(folhas_pagar2.columns)
folhas_pagar2.head()


(203, 40)
Index(['username', 'chefe de equipa', 'função', 'parceiro', 'Activações',
       'Correção autos anteriores', 'Correções autos anteriores_histórico',
       'Correções autos anteriores a aplicar', 'Total ativações',
       'Penalidades N', 'Penalidades_histórico', 'Penalidade total a aplicar',
       'Charge back N', 'Charge back_histórico', 'Charge back total a aplicar',
       'Ativação Negativa', 'Ativação Negativa Histórico',
       'Ativação Negativa total a aplicar', 'Melhoria Contínua', 'Valor',
       'VALOR COM FATOR', 'Correções a transitar', 'Penalidades a transitar',
       'Charge back a transitar', 'Ativações negativas a transitar',
       'Status CAV N-1', 'Status CAV N', 'Imputar ao CC do OBV_correções',
       'Imputar ao CC do OBV_penalidades', 'Imputar ao CC do OBV_charge back',
       'Imputar ao CC do OBV_ativ neg',
       'Correções a transitar para o mês seguinte',
       'Penalidade a transitar para o mês seguinte',
       'Charge back a transitar para

,username,chefe de equipa,função,parceiro,Activações,Correção autos anteriores,Correções autos anteriores_histórico,Correções autos anteriores a aplicar,Total ativações,Penalidades N,...,Imputar ao CC do OBV_ativ neg,Correções a transitar para o mês seguinte,Penalidade a transitar para o mês seguinte,Charge back a transitar para o mês seguinte,Ativações negativas a transitar para o mês seguinte,rede,Auto,CAV,Valores_transitar,Comentário
6,U80086295,U80037039,Operador,Heading,990.6,4.852174,0,4.852174,995.452174,0,...,0,0,0,0,0,ONE Prospect,Prospect,80086295,0,ok
7,U80047659,U80037039,Operador,Heading,774.6,0,0,0,774.6,-32,...,0,0,0,0,0,ONE Prospect,Prospect,80047659,0,ok
8,amargrodrigues,U80037039,Operador,Randstad,582.6,0,0,0,582.6,-32,...,0,0,0,0,0,ONE Prospect,Prospect,80122712,0,ok
9,jfcsilva,U80037039,Operador,Heading,111.75,0,0,0,111.75,-6,...,0,0,0,0,0,ONE Prospect,Prospect,70080648,0,ok
10,PJBARROSO,TLCOIMBRA,Operador,Heading,939.4,-5.12,0,-5.12,934.28,-32,...,0,0,0,0,0,ONE Prospect,Prospect,80079762,0,ok


In [85]:
#identificar linhas com eventos a transitar

#def correcoes (df, colacti, colauto, coltrans, lista):
def correcoes (df, colacti, colauto, coltrans):
  #df2 tem pessoas com activações, com correcções e com correcções a serem aplicadas
  df2=df.loc[(df[colacti]>0) & (df[colauto]!=0) & (df[coltrans]!=0),:]
  #df2 separa as pessoas em que se aplica a correcção total e aquelas que é preciso ratear
  df2["status"]=df2[colauto].where(abs(df2[colauto])>abs(df2[coltrans]), other="não ratear")
  df3=df2.loc[df2["status"]!="não ratear", :]

  return df3[["username", coltrans]]

df_ajustes=correcoes(folhas_pagar2, "Activações", "Correção autos anteriores", "Correções autos anteriores a aplicar")


#correcoes(folhas_pagar2, "Activações", "Ativação Negativa", "Ativação Negativa total a aplicar", ["username", \
#"Activações", "Ativação Negativa", "Ativação Negativa total a aplicar"])

<ipython-input-85-3d7d2e32ab8a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["status"]=df2[colauto].where(abs(df2[colauto])>abs(df2[coltrans]), other="não ratear")


In [87]:
#lista=df2.loc[df2["status"]!="não ratear", ["username", "Correções autos anteriores a aplicar"]].apply(lambda i: (i[0], i[1]), axis=1).tolist()

lista=df_ajustes.apply(lambda i: (i[0], i[1]), axis=1).tolist()

lista

[('sbmarante', -2), ('pspaulino', -1)]

In [74]:
dici_evento={"correcção": ["Correcções marcações", "Correcções autos anteriores"]}

def filtrar (df, evento, lista):
  colunas=dici_evento.get(evento)
  for i, j in lista:
    print(df.loc[df["Username operador"]==i, "Username operador"].count())


filtrar(folha_capit, "correcção", lista)



99
81
